In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from model import get_model
from dataset import get_dataset
import tensorflow_probability as tfp
from tqdm.notebook import tqdm

2023-02-14 20:39:55.373125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 20:39:55.545516: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
tf.config.list_physical_devices("GPU")

2023-02-14 20:40:00.293157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2023-02-14 20:40:00.371203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 20:40:00.371324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
base_model, model = get_model(224)
model.load_weights("model/model224.ckpt")
test_ds, val_ds = get_dataset(False, 224, 1, 16, True)

2023-02-14 20:40:00.404636: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 20:40:00.405203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 20:40:00.405345: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-14 20:40:00.405416: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative val

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
conv_idx = [
    i for i, l in enumerate(base_model.layers) if isinstance(l, tf.keras.layers.Conv2D)
]

In [5]:
def apply_mask(out, mask, weights):
    #return out * tf.cast(tf.logical_not(mask), tf.float32) + weights * tf.cast(
    #    mask, tf.float32
    #)
    return out * tf.cast(tf.logical_not(mask), tf.float32)

In [6]:
def call_wrapper(func, mask, weights):
    def wrapper(*args, **kwargs):
        return apply_mask(func(*args, **kwargs), mask, weights)

    return wrapper

In [7]:
def set_const(layer, mask, weights):
    if not "partial" in layer.name:
        layer.default_call = layer.call
        layer._name = f"partial_{layer.name}"
    layer.call = call_wrapper(layer.default_call, mask, weights)

In [8]:
def test():
    weights = tf.random.uniform(shape=[112, 112, 24], minval=0, maxval=4)
    mask = tf.random.uniform(shape=[112, 112, 24], minval=0, maxval=1) > 0.2
    set_const(base_model.layers[conv_idx[0]], mask, weights)
    random_input = tf.random.uniform(shape=[1, 224, 224, 3])
    out1 = base_model.layers[conv_idx[0]].call(random_input)
    out2 = base_model.layers[conv_idx[0]].default_call(random_input)
    print(tf.reduce_sum(tf.cast(out1 == out2, tf.float32) / (112 * 112 * 24)))

In [9]:
def get_activations(model):
    activations = []
    for j, (d, t) in enumerate(test_ds):
        if j == 40:
            break
        activations.append(model(d))
    return tf.concat(activations, 0)

In [10]:
for i in tqdm(conv_idx):
    m = tf.keras.Model(inputs=base_model.input, outputs=base_model.layers[i].output)
    activations = get_activations(m)
    stds = tf.math.reduce_std(activations, 0)
    means = tf.math.reduce_mean(activations, 0)
    t_min, t_cur, t_max = 0, 50, 100
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
    base_accuracy = model.evaluate(test_ds, verbose=0)[1]
    min_cutoff = 0.0
    print(f"BASELINE: {base_accuracy:.5f}")
    while t_max - t_min > 100 / 2**5:
        cutoff = tfp.stats.percentile(stds, t_cur)
        set_const(base_model.layers[i], stds < cutoff, means)
        model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
        out_accuracy = model.evaluate(test_ds, verbose=0)[1]
        if out_accuracy >= base_accuracy - 1e-3:
            t_min = t_cur
            min_cutoff = cutoff
            if out_accuracy > base_accuracy:
                base_accuracy = out_accuracy
        else:
            t_max = t_cur
        t_cur = (t_max - t_min) / 2 + t_min
        print(f"Accuracy: {out_accuracy:.5f}, threshold: {t_cur}%")
    print(f"Threshold: {min_cutoff:.5f} ({t_min}%)")
    set_const(base_model.layers[i], stds < min_cutoff, means)

  0%|          | 0/140 [00:00<?, ?it/s]

2023-02-14 20:40:15.847497: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-02-14 20:40:20.432842: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


BASELINE: 0.73780
Accuracy: 0.63900, threshold: 25.0%
Accuracy: 0.73220, threshold: 12.5%
Accuracy: 0.73840, threshold: 18.75%
Accuracy: 0.73660, threshold: 15.625%
Accuracy: 0.73820, threshold: 17.1875%
Threshold: 0.06052 (15.625%)
BASELINE: 0.73820
Accuracy: 0.33260, threshold: 25.0%
Accuracy: 0.46820, threshold: 12.5%
Accuracy: 0.59420, threshold: 6.25%
Accuracy: 0.73680, threshold: 3.125%
Accuracy: 0.73800, threshold: 4.6875%
Threshold: 0.15995 (3.125%)
BASELINE: 0.73800
Accuracy: 0.55240, threshold: 25.0%
Accuracy: 0.67380, threshold: 12.5%
Accuracy: 0.70840, threshold: 6.25%
Accuracy: 0.73100, threshold: 3.125%
Accuracy: 0.72700, threshold: 1.5625%
Threshold: 0.00000 (0%)
BASELINE: 0.73800
Accuracy: 0.70820, threshold: 25.0%
Accuracy: 0.73240, threshold: 12.5%
Accuracy: 0.73280, threshold: 6.25%
Accuracy: 0.73520, threshold: 3.125%
Accuracy: 0.73640, threshold: 1.5625%
Threshold: 0.00000 (0%)
BASELINE: 0.73800
Accuracy: 0.49020, threshold: 25.0%
Accuracy: 0.68120, threshold: 12.5

2023-02-14 20:50:09.680226: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.44GiB (rounded to 1541406720)requested by op Square
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-02-14 20:50:09.680261: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-02-14 20:50:09.680270: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 336, Chunks in use: 334. 84.0KiB allocated for chunks. 83.5KiB in use in bin. 15.0KiB client-requested in use in bin.
2023-02-14 20:50:09.680274: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 76, Chunks in use: 76. 50.0KiB allocated for chunks. 50.0KiB in use in bin. 44.9KiB client-requested in use in bin.
2023-02-14 20:50:09.680277: I tensorflow/tsl/framewo

ResourceExhaustedError: {{function_node __wrapped__Square_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Square]